In [1]:
# [Credits to: https://www.kaggle.com/code/rkuo2000/gpt2-german-recipes]
# Anpassungen durch T. Ekert 2024

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'german-recipes-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F132879%2F316218%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240409%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240409T104015Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3ee6decec57dfa418d371e3b0735425fe856936811feddee964e6364e44bc9caa3654c51a519cc606c52d8946b8b28380c33e252561b45b52e87c2f439c043ca7006f1557950ad2a5ba3dbcf700dd725896da2febb24584d0d4ca4a106872a508b612e0223423132c6d5667552cde4c26bfcecf93576f36513bdfbb5229e488ead0832f655a9c76ed5ea4d20397ac7e5be8b7aea25477bab460b2ff370f19135a799ee61f3a33bfc9c65d00df602823da178cc49e930ddd8d4e4b16db8930599b0d2ddf622d8da6d0102a191ac8446e7ab6aa417d8e26db25ae725cf8d7c431cdf4e25432b9a2e3d6ea1796faf8032aeaf5899895015d8845fd50a815c722bdf'

KAGGLE_INPUT_PATH='../data/kaggle/input'
KAGGLE_WORKING_PATH='../data/kaggle/working'
#KAGGLE_SYMLINK='kaggle'

shutil.rmtree(KAGGLE_INPUT_PATH, ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 4749666 bytes downloaded
Downloaded and uncompressed: german-recipes-dataset
Data source import complete.


# GPT-2 fine tuning with German Recipes

## Dataset: [German Recipes](https://www.kaggle.com/sterby/german-recipes-dataset)

## Read Dataset

In [4]:
import re
import json
from sklearn.model_selection import train_test_split

In [5]:
data_path = KAGGLE_INPUT_PATH + '/german-recipes-dataset/'

with open(data_path + 'recipes.json', encoding="utf-8") as f:
    data = json.load(f)

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w', encoding="utf-8")
    data = ''
    for texts in data_json:
        summary = str(texts['Instructions']).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [6]:
print(data[0].keys())

dict_keys(['Url', 'Instructions', 'Ingredients', 'Day', 'Name', 'Year', 'Month', 'Weekday'])


In [7]:
print(data[0])

{'Url': 'https://www.chefkoch.de/rezepte/185441079701305/', 'Instructions': 'Die Eier hart kochen. Dann pellen und mit einem Eierschneider in Scheiben schneiden. Den Reis halbgar kochen und zur Seite stellen. Die Wurst (Kolbász) in dünne Scheiben schneiden.Den Knoblauch abziehen und fein würfeln. Die Zwiebel schälen, fein hacken und in etwas Fett glasig braten. Knoblauch und Hackfleisch dazu geben und so lange braten, bis das Hackfleisch schön krümelig wird. Den eigenen Saft nicht ganz verkochen lassen. Die Fleischmasse mit Salz, Pfeffer und Paprikapulver würzen.Das Sauerkraut kurz durchspülen, ausdrücken und abtropfen lassen (damit es nicht zu sauer wird). Das Sauerkraut in einen Topf geben und mit dem Kümmel und den Lorbeerblättern vermischen. Ca. 30 Minuten unter Zugabe von wenig Wasser bei niedriger Stufe dünsten.Eine feuerfeste Form mit etwas Öl einfetten und den Boden dünn mit Sauerkraut belegen. Darauf Kolbász und die Hälfte der in Scheiben geschnittene Eier verteilen, dann eine

### Split Dataset

In [8]:
train_path = KAGGLE_WORKING_PATH + '/train_dataset.txt'
test_path = KAGGLE_WORKING_PATH + '/test_dataset.txt'

train, test = train_test_split(data,test_size=0.15)

build_text_files(train, train_path)
build_text_files(test, test_path)

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 10361
Test dataset length: 1829


## Tokenize Input Text

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("anonymous-german-nlp/german-gpt2")



## Load Dataset

In [10]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

C:\Users\thomase\miniconda3\envs\studientage-pytorch\lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


## Train Model
### Initialize Trainer with TrainingArguments with German-GPT2 model

In [17]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("anonymous-german-nlp/german-gpt2")
model_name = KAGGLE_WORKING_PATH + "/gpt2-gerchef"

training_args = TrainingArguments(
    output_dir= model_name, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    report_to="tensorboard"
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [18]:
from transformers import pipeline
import os
os.environ["WANDB_DISABLED"] = "true"

# Als Vergleich sehen wir uns zunächst die Ergebnisse an, die das Vor-Trainierte GPT-2 OHNE unser Fine-Tuning erreicht:

In [19]:
# Test behavior of our non-tuned "bare" pretrained model
dump_chef = pipeline('text-generation',model="anonymous-german-nlp/german-gpt2", tokenizer='anonymous-german-nlp/german-gpt2')


In [20]:
dump_chef('Die Nudeln Kochen, Fleisch anbraten')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Die Nudeln Kochen, Fleisch anbraten und paste bis salsa oder mit Fleischstücken vermischen.\n(415 g)\nMehl und Eier zusammenfetten und mit der Sauce beträufeln.\nGießen'}]

In [21]:
dump_chef('Erstelle ein Rezept mit Hähnchen.')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Erstelle ein Rezept mit Hähnchen.\nDas Rezept auf den Teller hauchen, so dass nichts mehr schief gehen kann (wenn ich fertig bin).\nFajitas bäckt, wie beim Frühstück.\nWenn die Vorspeise gut, gut'}]

# Nun trainieren wir das Netz für das Fine-Tuning

In [22]:
## uncomment the following 2 lines to re-train the model!

# trainer.train()
# trainer.save_model()

## Test Model

In [23]:

chef = pipeline('text-generation',model=model_name, tokenizer='anonymous-german-nlp/german-gpt2')

#result = chef('Zuerst Hähnchen')[0]['generated_text']

In [24]:
chef('Die Nudeln Kochen, Fleisch anbraten')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Die Nudeln Kochen, Fleisch anbraten und in Scheiben schneiden, die Mischung kochen, in die Pfanne geben, und das Öl in einer großen Pfanne für die Käseplatte geben.\nDen Teig für über Nacht aufbewahren.\n1\n'}]

In [25]:
chef('Zuerst Hähnchen')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Zuerst Hähnchen auf der Pizza oder der Pizza auf der Platte und dann auf die Teller und Backen.\n1/3 Tasse Wasser.\n1/3 Tasse Mehl (zugedeckt).\nGießen Sie das Mehl, geben Sie die Eier und'}]

In [26]:
chef('Der beste Weg, um einen Schokoladenkuchen zuzubereiten, ist')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Der beste Weg, um einen Schokoladenkuchen zuzubereiten, ist ein kleiner Snack.\nBeim Toastbrot kann man die Schokolade gar nicht loben.\nMit den beiden verschiedenen Schokoladenrezepten kann man die Vanille-Kakao-Rezept'}]

In [27]:
chef('Zuerst Hähnchen')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Zuerst Hähnchenkätzchen (4 Arten in nur 2 kg) und viele andere Kätzchen.\nEs ist sehr beliebt, so dass sie in einigen Schulen oder bei Familien spielen können (besonders die Frauen in den USA).\nWenn Sie Kätzchen mögen'

In [28]:
model_name

'../data/kaggle/working/gpt2-gerchef'